<a href="https://colab.research.google.com/github/thecodemancer/study-with-me/blob/main/python/food_classification_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🍽 Food Classification Model Predict
![ceviche.avif](https://t1.rg.ltmcdn.com/es/posts/7/4/1/ceviche_peruano_18147_orig.jpg)

## Overview

This model is trained to recognize 2023 food dishes from images. It is based on MobileNet V1.

## Input
This model takes images as input.

Inputs are expected to be 3-channel RGB color images of size 224 x 224, scaled to [0, 1].

## Output
This model outputs to image_classifier.

image_classifier: A probability vector of dimension 2024, corresponding to a background class and one of 2023 food dishes in the labelmap.

## Model architecture
For details of the model architecture, see MobileNet V1.

## Suitable usecases
This model is trained to recognize 2023 food dishes. The training set includes entrees, side dishes, desserts, snacks, etc.

## Unsuitable usecases
Do not use this model to determine whether an object is edible or not.
This model is not suitable for predicting the ingredients of a food dish. Do not use this model to predict allergen or nutrition information.

## Known limitations
This model assumes that its input image contains a well-cropped food dish. If a non-food image is input to the model or if the dish is not well-cropped, the output of the model may be meaningless.
This model was trained on a dataset skewed toward North American foods.

## The Code

### Import tensorflow and tensorhub libraries

In [313]:
# Import TF2 version
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

### The Model
Let's wrap a SavedModel (or a legacy TF1 Hub format) as a Keras Layer. This layer wraps a callable object for use as a Keras layer. The callable object can be passed directly, or be specified by a Python string with a handle that gets passed to hub.load(). This is the preferred API to load a TF2-style SavedModel from TF Hub into a Keras model.

In [314]:
# This is the model
m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')
m

### Import other libraries

In [315]:
# Import other libraries
import numpy as np
import pandas as pd
import cv2 #OpenCV 
from skimage import io

### 🍽 Dish's URL

In [316]:
mysterious_food_url = "https://t1.rg.ltmcdn.com/es/posts/7/4/1/ceviche_peruano_18147_orig.jpg" #ceviche
#mysterious_food_url = "https://imgmedia.buenazo.pe/1200x660/buenazo/original/2020/09/15/5f6108f5462f6040a51d2ac4.jpg" #arroz con pollo
#mysterious_food_url = "https://finde.latercera.com/wp-content/uploads/2020/07/empanadas-entrada-listo-.jpg" #empanada
#mysterious_food_url = "https://i0.wp.com/munchinmash.com/wp-content/uploads/2022/04/Fried-Chicken-Web-Thumb-.jpg" #Fried Chicken
#mysterious_food_url = "https://jameaperu.com/wp-content/uploads/2020/10/caldo-de-pollo_700x465.jpg" #Caldo de pollo
#mysterious_food_url = "https://www.inspiredtaste.net/wp-content/uploads/2019/03/Spaghetti-with-Meat-Sauce-Recipe-3-1200.jpg" #Spaguetti

<img src="https://t1.rg.ltmcdn.com/es/posts/7/4/1/ceviche_peruano_18147_orig.jpg" width="200px" height="150px" />
<img src="https://imgmedia.buenazo.pe/1200x660/buenazo/original/2020/09/15/5f6108f5462f6040a51d2ac4.jpg" width="200px" height="150px" />
<img src="https://finde.latercera.com/wp-content/uploads/2020/07/empanadas-entrada-listo-.jpg" width="200px" height="150px" />
<img src="https://i0.wp.com/munchinmash.com/wp-content/uploads/2022/04/Fried-Chicken-Web-Thumb-.jpg" width="200px" height="150px" />
<img src="https://jameaperu.com/wp-content/uploads/2020/10/caldo-de-pollo_700x465.jpg" width="200px" height="150px" />
<img src="https://www.inspiredtaste.net/wp-content/uploads/2019/03/Spaghetti-with-Meat-Sauce-Recipe-3-1200.jpg" width="200px" height="150px" />

### The Dataset

In [317]:
#The Dataset
labelmap_url = "https://www.gstatic.com/aihub/tfhub/labelmaps/aiy_food_V1_labelmap.csv"
input_shape = (224, 224)

#### Let's load it into memory

In [318]:
df = pd.read_csv(labelmap_url)
print(df.head())

   id               name
0   0     __background__
1   1            Chaudin
2   2         Bambalouni
3   3            Ghoriba
4   4  Mango sticky rice


#### Exporing the dataset

Let's see what do we have in this dataset. For that, we'll search according to some popular foods.

In [319]:
df['name']=df['name'].str.lower()

In [320]:
df[
   df['name'].str.contains("ceviche") |
   df['name'].str.contains("arroz")   |
   df['name'].str.contains("empanada")|
   df['name'].str.contains("chicken") |
   df['name'].str.contains("caldo de pollo")
   ].sort_values(by="name")

,id,name
945,945,arroz a la cubana
163,163,arroz chaufa
1394,1394,arroz con pollo
1907,1907,beggar's chicken
221,221,bolo de arroz
93,93,bon bon chicken
588,588,butter chicken
431,431,caldo de pollo
7,7,carrozza
1923,1923,ceviche


#### Printing the entire list of dishes in the dataset


In [321]:
#Printing the entire list of dishes in the dataset
classes = list(df["name"])
print(classes)

['__background__', 'chaudin', 'bambalouni', 'ghoriba', 'mango sticky rice', 'jianbing', 'aguachile', 'carrozza', 'miyan kuka', 'efo riro', 'ayam masak merah', 'chiffon pie', '/g/11b8_rxx4d', 'riso patate e cozze', 'bazin', 'black bottom pie', 'palóc soup', 'sailor sandwich', 'tuwo shinkafa', 'carne a la tampiqueña', 'pastel azteca', 'fujian red wine chicken', 'boeber', 'lady baltimore cake', 'yam khai dao', 'texas tommy', 'har cheong gai', 'kolokythopita', 'karydopita', 'rinflajš', 'hainanese curry rice', 'sonoran hot dog', '/g/11cfty6q3', 'afghani burger', 'teochew porridge', 'minestra di ceci', 'pastrami on rye', 'roast beef sandwich', 'chahan', 'ekuru', 'sciusceddu', 'breakfast burrito', '/g/11dyjj24g', 'sausage stroganoff', 'roti jala', 'pirao', 'casatiello', 'khanom tan', 'muamba chicken', 'dobradinha', 'bruckfleisch', 'molote', 'spongata', 'funge', '/g/1212ghsj', 'köttbullar', "ka'ak", 'papet vaudois', '/g/12148tdg', 'prosciutto di norcia', 'malloreddus', '/g/1214g6v_', 'pannenko

### Load the dish's photo and convert it into a numpy array

In [322]:
#image = np.asarray(cv2.imread(mysterious_food_url), dtype="float") # After reading in the image data will be stored in a cv::Mat object.
image = np.asarray(io.imread(mysterious_food_url), dtype="float")
image.shape

(588, 900, 3)

In [323]:
image

array([[[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       [[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       [[253., 253., 253.],
        [253., 253., 253.],
        [253., 253., 253.],
        ...,
        [239., 231., 242.],
        [239., 231., 242.],
        [239., 231., 242.]],

       ...,

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 239., 245.],
        ...,
        [204., 214., 224.],
        [201., 211., 221.],
        [207., 217., 227.]],

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 239., 245.],
        ...,
        [205., 215., 225.],
        [205., 215., 225.],
        [208., 218., 228.]],

       [[239., 238., 244.],
        [240., 239., 245.],
        [240., 2

### Data Processing

#### Resizing

In [324]:
image = cv2.resize(image, dsize=input_shape, interpolation=cv2.INTER_CUBIC)
image

array([[[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.        , 231.        , 242.        ]],

       [[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.        , 232.1038208 , 239.7923584 ]],

       [[253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        [253.        , 253.        , 253.        ],
        ...,
        [240.        , 233.        , 240.        ],
        [240.        , 233.        , 240.        ],
        [239.00000918, 232.00000918, 239.95032656]],

       ...,

      

#### Scale values to [0, 1].

In [325]:
# Scale values to [0, 1].
image = image / image.max()
image

array([[[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279213, 0.8532426 , 0.8938732 ]],

       [[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279213, 0.85731978, 0.88571886]],

       [[0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        [0.9345038 , 0.9345038 , 0.9345038 ],
        ...,
        [0.88648582, 0.86062998, 0.88648582],
        [0.88648582, 0.86062998, 0.88648582],
        [0.88279216, 0.85693633, 0.88630234]],

       ...,

       [[0.88092054, 0.87722685, 0.899389  ],
        [0.87504096, 0.87134727, 0.89350942],
        [0.87712874, 0

#### The model expects an input of (?, 224, 224, 3).

In [326]:
# The model expects an input of (?, 224, 224, 3).
images = np.expand_dims(image, 0)
images.shape

(1, 224, 224, 3)

#### Let's see how we go

In [327]:
images

array([[[[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279213, 0.8532426 , 0.8938732 ]],

        [[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279213, 0.85731978, 0.88571886]],

        [[0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         [0.9345038 , 0.9345038 , 0.9345038 ],
         ...,
         [0.88648582, 0.86062998, 0.88648582],
         [0.88648582, 0.86062998, 0.88648582],
         [0.88279216, 0.85693633, 0.88630234]],

        ...,

        [[0.88092054, 0.87722685, 0.899389  ],
         [0.87504096, 0.87134727, 0.89350942]

### Let's pass our numpy array to our Keras model

In [328]:
output = m(images)
type(output)

tensorflow.python.framework.ops.EagerTensor

### Now we have a Tensor!

In [329]:
print(output)

tf.Tensor(
[[1.3755466e-11 3.1406493e-08 4.5174867e-10 ... 2.1996420e-09
  8.2277585e-10 9.7932712e-12]], shape=(1, 2024), dtype=float32)


In [330]:
output.numpy()

array([[1.3755466e-11, 3.1406493e-08, 4.5174867e-10, ..., 2.1996420e-09,
        8.2277585e-10, 9.7932712e-12]], dtype=float32)

### Let's find the max value in the numpy array version of this tensor

In [331]:
predicted_index = output.numpy().argmax()
print(predicted_index)

1923


### The prediction

In [332]:
# The prediction
print("And voilà! this is what you're eating: ", classes[predicted_index])
print("🍽😋")

And voilà! this is what you're eating:  ceviche
🍽😋


---
If you made it this far, follow [David Regalado](https://beacons.ai/davidregalado) for more code!